In [1]:
import numpy as np , pandas as pd
import pyodbc
import statistics
from tqdm import tqdm
import time
import math
import datetime
from datetime import datetime
from configparser import ConfigParser
import pyodbc


In [2]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [3]:
# # Changed Code for audit integration

# conn.execute("""insert into FINCORE_DB.Fincore_Analytics.AUDIT_INTGN (JOB_RUN_ID,JOB_NAME,JOB_START_DATE,JOB_STATUS,TARGET_TBL,SOURCE_TBL,SOURCE_MAX_DATE)values
# (COALESCE((select (max(JOB_RUN_ID)+1) from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN),1), 'RISK_REPORT',GETDATE(),'Error','RISK_REPORT','RISK_TRANSACTION_SUMMARY_STG',(select max(update_date) from FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG))""")

# job_id = pd.read_sql_query("select max(JOB_RUN_ID) as job_run_id from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN",conn)
# job_id = job_id.iloc[0,0]
# conn.commit()



## Populating RISK_REPORT_SUMMARY_STG  

In [4]:
# FUNCTION TO ROUND THE VALUE.
import math
def round_func(num):
    if num-math.floor(num)<0.5:
        return math.floor(num)
    return math.ceil(num)

In [5]:
# pyodbc.drivers()
# conn = pyodbc.connect('DRIVER=SQL Server;SERVER=172.16.34.15;PORT=1433;DATABASE=FINCORE_DB;UID=Fincore_Analytics_USR;PWD=F!NCore@123;TDS_Version=7.2;')
# cursor = conn.cursor()
summary_STG_table = pd.read_sql_query("select * from Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG with(nolock)", conn)




In [6]:
RISK_REPORT_SUMMARY_STG = summary_STG_table[['BATCH_ID','REPORT_ID','YEAR_REPORT','MONTH_REPORT','REPORT_TYPE','CREATE_DATE','UPDATE_DATE']]
RISK_REPORT_SUMMARY_STG

BATCH_ID  REPORT_ID  YEAR_REPORT  MONTH_REPORT REPORT_TYPE  \
0      109702     107413         2022            11         STR   
1      110814     108512         2022            12         STR   
2      113304     110813         2022            12         STR   
3      116402     113515         2022            12         STR   
4      122002     118613         2023             1         STR   
..        ...        ...          ...           ...         ...   
247    122203     118814         2023             1         STR   
248    135805     133416         2023             4         NTR   
249    115401     112712         2022            12         STR   
250    115401     112712         2022            12         STR   
251    111703     109226         2022            12         STR   

                     CREATE_DATE                  UPDATE_DATE  
0    2023-05-15 20:01:52.2466667  2023-05-15 20:01:52.2466667  
1    2023-05-15 20:01:52.2466667  2023-05-15 20:01:52.2466667  
2    2023-05-15 20:01:52.2466667  2023-05-15 20:01:52.2466667  
3    2023-05-15 20:01:52.2466667  2023-05-15 20:01:52.2466667  
4    2023-05-15 20:01:52.2466667  2023-05-15 20:01:52.2466667  
..                           ...                          ...  
247  2023-05-15 20:01:52.6266667  2023-05-15 20:01:52.6266667  
248  2023-05-15 20:01:52.6266667  2023-05-15 20:01:52.6266667  
249  2023-05-15 20:01:52.7700000  2023-05-15 20:01:52.7700000  
250  2023-05-15 20:01:52.7700000  2023-05-15 20:01:52.7700000  
251  2023-05-15 20:01:52.9800000  2023-05-15 20:01:52.9800000  

[252 rows x 7 columns]

In [7]:
Tran_risk_table = pd.read_sql_query("select * from Fincore_Analytics.RISK_TRANSACTION with(nolock)", conn)

Tran_risk_table

BATCH_ID  REPORT_ID REPORT_TYPE  RISK_SCORE                  CREATE_DATE
0      102901     100712         STR          10  2023-05-18 10:41:31.7233333
1      104102     102012         STR           4  2023-05-18 10:41:31.7400000
2      104104     102013         STR           4  2023-05-18 10:41:31.7566667
3      105201     103112       CBWTR           3  2023-05-18 10:41:31.7600000
4      105306     103216         NTR           3  2023-05-18 10:41:31.7733333
..        ...        ...         ...         ...                          ...
177    137501     135112         CTR           3  2023-05-18 10:41:34.3300000
178    138101     135712         CTR           3  2023-05-18 10:41:34.3533333
179    138201     135812         STR           3  2023-05-18 10:41:34.3666667
180    138203     135814         CTR           3  2023-05-18 10:41:34.3833333
181    141201     138712         CTR          10  2023-05-18 10:41:34.4166667

[182 rows x 5 columns]

In [8]:
# RISK_REPORT_SUMMARY_STG.drop_duplicates(inplace = True)

In [9]:
# RISK_REPORT_SUMMARY_STG

In [10]:
# sr_list_temp=RISK_REPORT_SUMMARY_STG.values.tolist()

In [11]:
# sr_list_temp

### Inserting data in RISK_REPORT_SUMMARY_STG

In [12]:
from tqdm import tqdm

### Creating Table RISK_REPORT Using Tables RISK_TRANSACTION, RISK_PERSON and RISK_GOS

In [13]:
# MERGING RISK_TRANSACTION_SUMMARY_STG AND RISK_TRANSACTION
Tran_risk_table = summary_STG_table.merge(Tran_risk_table, how = 'left', left_on= ['BATCH_ID','REPORT_ID'],right_on = ['BATCH_ID','REPORT_ID'])

In [14]:
Tran_risk_table = Tran_risk_table.drop(['CREATE_DATE_x','YEAR_REPORT', 'MONTH_REPORT', 'REPORT_TYPE_y','PEER_GROUP_ID', 'OCCUPATION_BUSINESS', 'TRAN_TYP','CR_DR_DEP_WITH',   'TRAN_AMNT', 'CREATE_DATE_y', 'UPDATE_DATE'], axis=1)

In [15]:
Tran_risk_table.rename(columns = {'REPORT_TYPE_x':'REPORT_TYPE'},inplace = True)

In [16]:
Tran_risk_table.drop_duplicates(inplace = True)

In [17]:
Tran_risk_table.rename(columns = {'RISK_SCORE':'TRANSACTION_SCORE'},inplace = True)

In [18]:
Tran_risk_table

ID_  BATCH_ID  REPORT_ID REPORT_TYPE  FINGATE_ACCOUNT_ID TRANSACTION_ID  \
0      1    109702     107413         STR               84809        1000256   
1      2    110814     108512         STR               85505    78784478747   
2      3    113304     110813         STR               88201       52003149   
3      4    116402     113515         STR               91207  1728937819273   
4      5    122002     118613         STR              103501        9763462   
..   ...       ...        ...         ...                 ...            ...   
247  248    122203     118814         STR              103704      234242523   
248  249    135805     133416         NTR             7807404      54255253K   
249  250    115401     112712         STR               90601         100802   
250  251    115401     112712         STR               90601         100803   
251  252    111703     109226         STR               86319        2563001   

      TRAN_DATE LOCATION_PINCODE  ENTITY_ID ENTITY_TYPE  TRANSACTION_SCORE  
0    2022-11-14             None         24           I                  2  
1    2022-11-29             None          4           O                  3  
2    2021-06-15             None         58           I                  2  
3    2022-12-12             None         24           O                  2  
4    2022-12-06             None        229           I                  3  
..          ...              ...        ...         ...                ...  
247  2022-11-30             None         35           O                  6  
248  2023-04-01             None        104           O                  3  
249  2020-07-15             None         25           I                  2  
250  2021-02-03             None         25           I                  2  
251  2022-05-05             None         33           I                  2  

[252 rows x 11 columns]

In [19]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [20]:
Entity_risk_table = pd.read_sql_query("select * from Fincore_Analytics.RISK_PERSON with(nolock)", conn)
conn.close()


In [21]:
Entity_risk_table['ENTITY_TYPE']='I'

In [22]:
Entity_risk_table

PERSON_MASTER_ID        PEER_GROUP_ID  RISK_SCORE  \
0                   1  Government Sector_1           7   
1                   2  Government Sector_1           8   
2                   3  Government Sector_1           8   
3                   4  Government Sector_1           8   
4                   5  Government Sector_1          10   
..                ...                  ...         ...   
423               536     OTHERS_CHANGED_9           7   
424               537             Others_0           1   
425               538             Others_0           1   
426               539             Others_0           1   
427               540             Others_0           1   

                CREATE_DATE ENTITY_TYPE  
0   2023-05-12 13:41:22.317           I  
1   2023-05-12 13:41:22.320           I  
2   2023-05-12 13:41:22.320           I  
3   2023-05-12 13:41:22.320           I  
4   2023-05-12 13:41:15.783           I  
..                      ...         ...  
423 2023-05-12 13:41:22.833           I  
424 2023-05-12 13:41:22.833           I  
425 2023-05-12 13:41:22.833           I  
426 2023-05-12 13:41:22.837           I  
427 2023-05-12 13:41:22.837           I  

[428 rows x 5 columns]

In [23]:
Tran_risk_table[Tran_risk_table['ENTITY_TYPE']=='O']

ID_  BATCH_ID  REPORT_ID REPORT_TYPE  FINGATE_ACCOUNT_ID TRANSACTION_ID  \
1      2    110814     108512         STR               85505    78784478747   
3      4    116402     113515         STR               91207  1728937819273   
5      6    122202     118813         STR              103701        1234112   
7      8    122203     118814         STR              103703        2352352   
8      9    122203     118814         STR              103704         545433   
..   ...       ...        ...         ...                 ...            ...   
214  215    120101     116813         STR              100404    46647478787   
244  245    122202     118813         STR              103701       12324141   
246  247    122203     118814         STR              103703        3964641   
247  248    122203     118814         STR              103704      234242523   
248  249    135805     133416         NTR             7807404      54255253K   

      TRAN_DATE LOCATION_PINCODE  ENTITY_ID ENTITY_TYPE  TRANSACTION_SCORE  
1    2022-11-29             None          4           O                  3  
3    2022-12-12             None         24           O                  2  
5    2022-12-06             None         34           O                  3  
7    2022-12-05             None         34           O                  6  
8    2022-12-06             None         35           O                  6  
..          ...              ...        ...         ...                ...  
214  2022-08-14           700086         37           O                  3  
244  2022-12-06             None         34           O                  3  
246  2022-12-05             None         34           O                  6  
247  2022-11-30             None         35           O                  6  
248  2023-04-01             None        104           O                  3  

[78 rows x 11 columns]

In [24]:
# MERGING RISK_TRANSACTION_SUMMARY_STG AND ENTITY_RISK
merge_TR_ER = Tran_risk_table.merge(Entity_risk_table, how = 'left', left_on= ['ENTITY_ID','ENTITY_TYPE'],right_on = ['PERSON_MASTER_ID','ENTITY_TYPE'])
merge_TR_ER

ID_  BATCH_ID  REPORT_ID REPORT_TYPE  FINGATE_ACCOUNT_ID TRANSACTION_ID  \
0      1    109702     107413         STR               84809        1000256   
1      2    110814     108512         STR               85505    78784478747   
2      3    113304     110813         STR               88201       52003149   
3      4    116402     113515         STR               91207  1728937819273   
4      5    122002     118613         STR              103501        9763462   
..   ...       ...        ...         ...                 ...            ...   
247  248    122203     118814         STR              103704      234242523   
248  249    135805     133416         NTR             7807404      54255253K   
249  250    115401     112712         STR               90601         100802   
250  251    115401     112712         STR               90601         100803   
251  252    111703     109226         STR               86319        2563001   

      TRAN_DATE LOCATION_PINCODE  ENTITY_ID ENTITY_TYPE  TRANSACTION_SCORE  \
0    2022-11-14             None         24           I                  2   
1    2022-11-29             None          4           O                  3   
2    2021-06-15             None         58           I                  2   
3    2022-12-12             None         24           O                  2   
4    2022-12-06             None        229           I                  3   
..          ...              ...        ...         ...                ...   
247  2022-11-30             None         35           O                  6   
248  2023-04-01             None        104           O                  3   
249  2020-07-15             None         25           I                  2   
250  2021-02-03             None         25           I                  2   
251  2022-05-05             None         33           I                  2   

     PERSON_MASTER_ID        PEER_GROUP_ID  RISK_SCORE             CREATE_DATE  
0                24.0     Private Sector_0         8.0 2023-05-12 13:41:22.340  
1                 NaN                  NaN         NaN                     NaT  
2                58.0  Government Sector_1         8.0 2023-05-12 13:41:22.367  
3                 NaN                  NaN         NaN                     NaT  
4               229.0     OTHERS_CHANGED_9         7.0 2023-05-12 13:41:22.527  
..                ...                  ...         ...                     ...  
247               NaN                  NaN         NaN                     NaT  
248               NaN                  NaN         NaN                     NaT  
249              25.0  Government Sector_1         8.0 2023-05-12 13:41:22.343  
250              25.0  Government Sector_1         8.0 2023-05-12 13:41:22.343  
251              33.0     Private Sector_0         8.0 2023-05-12 13:41:22.350  

[252 rows x 15 columns]

In [25]:
merge_TR_ER.rename(columns = {'RISK_SCORE' :'RISK_SCORE_PERSON'},inplace=True)

In [26]:
merge_TR_ER.drop(columns = ['PERSON_MASTER_ID','PEER_GROUP_ID','CREATE_DATE'],inplace = True)

In [27]:
merge_TR_ER.drop_duplicates(inplace = True)

In [28]:
#  merge_TR_ER = merge_TR_ER[['BATCH_ID','REPORT_ID','REPORT_TYPE','RISK_SCORE','CREATE_DATE']]

In [29]:
 merge_TR_ER

ID_  BATCH_ID  REPORT_ID REPORT_TYPE  FINGATE_ACCOUNT_ID TRANSACTION_ID  \
0      1    109702     107413         STR               84809        1000256   
1      2    110814     108512         STR               85505    78784478747   
2      3    113304     110813         STR               88201       52003149   
3      4    116402     113515         STR               91207  1728937819273   
4      5    122002     118613         STR              103501        9763462   
..   ...       ...        ...         ...                 ...            ...   
247  248    122203     118814         STR              103704      234242523   
248  249    135805     133416         NTR             7807404      54255253K   
249  250    115401     112712         STR               90601         100802   
250  251    115401     112712         STR               90601         100803   
251  252    111703     109226         STR               86319        2563001   

      TRAN_DATE LOCATION_PINCODE  ENTITY_ID ENTITY_TYPE  TRANSACTION_SCORE  \
0    2022-11-14             None         24           I                  2   
1    2022-11-29             None          4           O                  3   
2    2021-06-15             None         58           I                  2   
3    2022-12-12             None         24           O                  2   
4    2022-12-06             None        229           I                  3   
..          ...              ...        ...         ...                ...   
247  2022-11-30             None         35           O                  6   
248  2023-04-01             None        104           O                  3   
249  2020-07-15             None         25           I                  2   
250  2021-02-03             None         25           I                  2   
251  2022-05-05             None         33           I                  2   

     RISK_SCORE_PERSON  
0                  8.0  
1                  NaN  
2                  8.0  
3                  NaN  
4                  7.0  
..                 ...  
247                NaN  
248                NaN  
249                8.0  
250                8.0  
251                8.0  

[252 rows x 12 columns]

In [30]:
temp = merge_TR_ER

In [31]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [32]:
Org_risk_table = pd.read_sql_query("select * from Fincore_Analytics.RISK_ORGANIZATION with(nolock)", conn)
conn.close()

In [33]:
Org_risk_table['ENTITY_TYPE']='O'

In [34]:
Org_risk_table

ORGANIZATION_MASTER_ID     PEER_GROUP_ID  RISK_SCORE RISK_RULE_LIST  \
0                        1    Not Declared_0           8           None   
1                        2    Not Declared_0           6           None   
2                        3    Not Declared_0           6           None   
3                        4    Not Declared_0           7           None   
4                        5    Not Declared_0           6           None   
..                     ...               ...         ...            ...   
80                     111    Not Declared_0           8           None   
81                     112    Not Declared_0           4           None   
82                     114    Not Declared_0           4           None   
83                     115  OTHERS_CHANGED_4           8           None   
84                     116    Not Declared_0           7           None   

               CREATE_DATE ENTITY_TYPE  
0  2023-05-15 19:39:06.187           O  
1  2023-05-15 19:39:06.187           O  
2  2023-05-15 19:39:06.190           O  
3  2023-05-15 19:39:06.190           O  
4  2023-05-15 19:39:06.190           O  
..                     ...         ...  
80 2023-05-15 19:39:06.243           O  
81 2023-05-15 19:39:06.243           O  
82 2023-05-15 19:39:06.243           O  
83 2023-05-15 19:39:06.243           O  
84 2023-05-15 19:39:06.243           O  

[85 rows x 6 columns]

In [35]:
merge_TR_ER = merge_TR_ER.merge(Org_risk_table, how = 'left', left_on= ['ENTITY_ID','ENTITY_TYPE'],right_on = ['ORGANIZATION_MASTER_ID','ENTITY_TYPE'])


In [36]:
merge_TR_ER.drop(columns = ['PEER_GROUP_ID','RISK_RULE_LIST','CREATE_DATE','ORGANIZATION_MASTER_ID'],inplace = True)

In [37]:
merge_TR_ER.rename(columns = {'RISK_SCORE':'ORG_SCORE'},inplace = True)

In [38]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [39]:
GOS_risk_table = pd.read_sql_query("select * from FINCORE_DB.Fincore_Analytics.RISK_GOS with(nolock)", conn)


In [40]:
GOS_risk_table.drop(columns = ['CREATE_DATE'],inplace = True)

In [41]:
GOS_risk_table

Empty DataFrame
Columns: [BATCH_ID, REPORT_ID, RISK_SCORE]
Index: []

In [42]:
all_data = merge_TR_ER.merge(GOS_risk_table, how= 'left',left_on = ['BATCH_ID','REPORT_ID'],right_on = ['BATCH_ID','REPORT_ID'])

In [43]:
all_data.rename(columns = {'RISK_SCORE' :'RISK_SCORE_GOS'},inplace=True)

In [44]:
all_data.head()

ID_  BATCH_ID  REPORT_ID REPORT_TYPE  FINGATE_ACCOUNT_ID TRANSACTION_ID  \
0    1    109702     107413         STR               84809        1000256   
1    2    110814     108512         STR               85505    78784478747   
2    3    113304     110813         STR               88201       52003149   
3    4    116402     113515         STR               91207  1728937819273   
4    5    122002     118613         STR              103501        9763462   

    TRAN_DATE LOCATION_PINCODE  ENTITY_ID ENTITY_TYPE  TRANSACTION_SCORE  \
0  2022-11-14             None         24           I                  2   
1  2022-11-29             None          4           O                  3   
2  2021-06-15             None         58           I                  2   
3  2022-12-12             None         24           O                  2   
4  2022-12-06             None        229           I                  3   

   RISK_SCORE_PERSON  ORG_SCORE RISK_SCORE_GOS  
0                8.0        NaN            NaN  
1                NaN        7.0            NaN  
2                8.0        NaN            NaN  
3                NaN        6.0            NaN  
4                7.0        NaN            NaN

In [45]:
all_data['RISK_SCORE_GOS'].fillna(0,inplace=True)

In [46]:
all_data

ID_  BATCH_ID  REPORT_ID REPORT_TYPE  FINGATE_ACCOUNT_ID TRANSACTION_ID  \
0      1    109702     107413         STR               84809        1000256   
1      2    110814     108512         STR               85505    78784478747   
2      3    113304     110813         STR               88201       52003149   
3      4    116402     113515         STR               91207  1728937819273   
4      5    122002     118613         STR              103501        9763462   
..   ...       ...        ...         ...                 ...            ...   
247  248    122203     118814         STR              103704      234242523   
248  249    135805     133416         NTR             7807404      54255253K   
249  250    115401     112712         STR               90601         100802   
250  251    115401     112712         STR               90601         100803   
251  252    111703     109226         STR               86319        2563001   

      TRAN_DATE LOCATION_PINCODE  ENTITY_ID ENTITY_TYPE  TRANSACTION_SCORE  \
0    2022-11-14             None         24           I                  2   
1    2022-11-29             None          4           O                  3   
2    2021-06-15             None         58           I                  2   
3    2022-12-12             None         24           O                  2   
4    2022-12-06             None        229           I                  3   
..          ...              ...        ...         ...                ...   
247  2022-11-30             None         35           O                  6   
248  2023-04-01             None        104           O                  3   
249  2020-07-15             None         25           I                  2   
250  2021-02-03             None         25           I                  2   
251  2022-05-05             None         33           I                  2   

     RISK_SCORE_PERSON  ORG_SCORE  RISK_SCORE_GOS  
0                  8.0        NaN               0  
1                  NaN        7.0               0  
2                  8.0        NaN               0  
3                  NaN        6.0               0  
4                  7.0        NaN               0  
..                 ...        ...             ...  
247                NaN        6.0               0  
248                NaN        5.0               0  
249                8.0        NaN               0  
250                8.0        NaN               0  
251                8.0        NaN               0  

[252 rows x 14 columns]

In [47]:
# report_risk_rules_table = pd.read_sql_query("select * from FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES with(nolock)", conn)
# report_risk_rules_table

In [48]:
# all_data = all_data.merge(report_risk_rules_table, how= 'left',left_on = ['BATCH_ID','REPORT_ID'],right_on = ['BATCH_ID','REPORT_ID'])

In [49]:
# all_data[all_data['REPORT_ID']==46316]

In [50]:
# all_data.rename(columns = {'RISK_SCORE_STATIC' :'RISK_SCORE_REPORT_RULES'},inplace=True)

In [51]:
# all_data[all_data['REPORT_ID']==46316]

In [52]:
all_data['ENTITY_RISK']  = all_data[['RISK_SCORE_PERSON','ORG_SCORE']].bfill(axis=1).iloc[:, 0]

In [53]:
all_data.drop(columns = ['RISK_SCORE_PERSON','ORG_SCORE'],inplace = True)

In [54]:
all_data.drop_duplicates(inplace= True)

In [55]:
all_data['TRANSACTION_SCORE'].fillna(0,inplace =True)
all_data['RISK_SCORE_GOS'].fillna(0,inplace =True)
all_data['ENTITY_RISK'].fillna(0,inplace =True)

In [56]:
all_data

ID_  BATCH_ID  REPORT_ID REPORT_TYPE  FINGATE_ACCOUNT_ID TRANSACTION_ID  \
0      1    109702     107413         STR               84809        1000256   
1      2    110814     108512         STR               85505    78784478747   
2      3    113304     110813         STR               88201       52003149   
3      4    116402     113515         STR               91207  1728937819273   
4      5    122002     118613         STR              103501        9763462   
..   ...       ...        ...         ...                 ...            ...   
247  248    122203     118814         STR              103704      234242523   
248  249    135805     133416         NTR             7807404      54255253K   
249  250    115401     112712         STR               90601         100802   
250  251    115401     112712         STR               90601         100803   
251  252    111703     109226         STR               86319        2563001   

      TRAN_DATE LOCATION_PINCODE  ENTITY_ID ENTITY_TYPE  TRANSACTION_SCORE  \
0    2022-11-14             None         24           I                  2   
1    2022-11-29             None          4           O                  3   
2    2021-06-15             None         58           I                  2   
3    2022-12-12             None         24           O                  2   
4    2022-12-06             None        229           I                  3   
..          ...              ...        ...         ...                ...   
247  2022-11-30             None         35           O                  6   
248  2023-04-01             None        104           O                  3   
249  2020-07-15             None         25           I                  2   
250  2021-02-03             None         25           I                  2   
251  2022-05-05             None         33           I                  2   

     RISK_SCORE_GOS  ENTITY_RISK  
0                 0          8.0  
1                 0          7.0  
2                 0          8.0  
3                 0          6.0  
4                 0          7.0  
..              ...          ...  
247               0          6.0  
248               0          5.0  
249               0          8.0  
250               0          8.0  
251               0          8.0  

[252 rows x 13 columns]

In [57]:
all_data['FINAL_SCORE']= (((all_data['TRANSACTION_SCORE']**2)+(all_data['RISK_SCORE_GOS']**2)+(all_data['ENTITY_RISK']**2))/(all_data['TRANSACTION_SCORE']+all_data['RISK_SCORE_GOS']+all_data['ENTITY_RISK']))

In [58]:
all_data['FINAL_SCORE'].fillna(0,inplace  =True)

In [59]:
all_data

ID_  BATCH_ID  REPORT_ID REPORT_TYPE  FINGATE_ACCOUNT_ID TRANSACTION_ID  \
0      1    109702     107413         STR               84809        1000256   
1      2    110814     108512         STR               85505    78784478747   
2      3    113304     110813         STR               88201       52003149   
3      4    116402     113515         STR               91207  1728937819273   
4      5    122002     118613         STR              103501        9763462   
..   ...       ...        ...         ...                 ...            ...   
247  248    122203     118814         STR              103704      234242523   
248  249    135805     133416         NTR             7807404      54255253K   
249  250    115401     112712         STR               90601         100802   
250  251    115401     112712         STR               90601         100803   
251  252    111703     109226         STR               86319        2563001   

      TRAN_DATE LOCATION_PINCODE  ENTITY_ID ENTITY_TYPE  TRANSACTION_SCORE  \
0    2022-11-14             None         24           I                  2   
1    2022-11-29             None          4           O                  3   
2    2021-06-15             None         58           I                  2   
3    2022-12-12             None         24           O                  2   
4    2022-12-06             None        229           I                  3   
..          ...              ...        ...         ...                ...   
247  2022-11-30             None         35           O                  6   
248  2023-04-01             None        104           O                  3   
249  2020-07-15             None         25           I                  2   
250  2021-02-03             None         25           I                  2   
251  2022-05-05             None         33           I                  2   

     RISK_SCORE_GOS  ENTITY_RISK  FINAL_SCORE  
0                 0          8.0         6.80  
1                 0          7.0         5.80  
2                 0          8.0         6.80  
3                 0          6.0         5.00  
4                 0          7.0         5.80  
..              ...          ...          ...  
247               0          6.0         6.00  
248               0          5.0         4.25  
249               0          8.0         6.80  
250               0          8.0         6.80  
251               0          8.0         6.80  

[252 rows x 14 columns]

In [60]:
all_data.rename(columns = {'final_score':'RISK_SCORE'}, inplace = True)

In [61]:
all_data.drop(columns = ['TRANSACTION_SCORE','ENTITY_ID','RISK_SCORE_GOS','ENTITY_RISK','ENTITY_TYPE'],inplace = True)

In [62]:
all_data

ID_  BATCH_ID  REPORT_ID REPORT_TYPE  FINGATE_ACCOUNT_ID TRANSACTION_ID  \
0      1    109702     107413         STR               84809        1000256   
1      2    110814     108512         STR               85505    78784478747   
2      3    113304     110813         STR               88201       52003149   
3      4    116402     113515         STR               91207  1728937819273   
4      5    122002     118613         STR              103501        9763462   
..   ...       ...        ...         ...                 ...            ...   
247  248    122203     118814         STR              103704      234242523   
248  249    135805     133416         NTR             7807404      54255253K   
249  250    115401     112712         STR               90601         100802   
250  251    115401     112712         STR               90601         100803   
251  252    111703     109226         STR               86319        2563001   

      TRAN_DATE LOCATION_PINCODE  FINAL_SCORE  
0    2022-11-14             None         6.80  
1    2022-11-29             None         5.80  
2    2021-06-15             None         6.80  
3    2022-12-12             None         5.00  
4    2022-12-06             None         5.80  
..          ...              ...          ...  
247  2022-11-30             None         6.00  
248  2023-04-01             None         4.25  
249  2020-07-15             None         6.80  
250  2021-02-03             None         6.80  
251  2022-05-05             None         6.80  

[252 rows x 9 columns]

In [63]:
all_data  = all_data.groupby(['BATCH_ID','REPORT_ID','REPORT_TYPE']).mean()

In [64]:
all_data.reset_index(inplace = True)

In [65]:
all_data.shape

(182, 6)

In [66]:
# REPORTS FLAGGED IN AUTO HIGH RULES.
auto_high = pd.read_sql_query('''select REPORT_ID from Fincore_Analytics.RISK_REPORT_RULES A
inner join Fincore_Analytics.RISK_LKP_RULES B 
on A.RULE_NO = B.RULE_NO
where B.PERSON_TYPE = 'Report' and B.RISK_TYPE = 'Auto High Rules '  ''',conn)

In [67]:
auto_high = auto_high['REPORT_ID'].to_list()

In [68]:
# REMOVING REPORTS FLAGGED IN AUTO HIGH RULES .
all_data = all_data[~all_data['REPORT_ID'].isin(auto_high)]

In [69]:
all_data['FINAL_SCORE'] = all_data['FINAL_SCORE'].astype('float64')

In [70]:
all_data['FINAL_SCORE'] = all_data['FINAL_SCORE'].apply(lambda x: round_func(x)) 

In [71]:
config = ConfigParser()

config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [72]:
# UPDATING THE RISK_REPORT FOR WEIGHTAGE PART
for ind,row in all_data.iterrows():
#     print(row['BATCH_ID'])
    
    cursor.execute("""UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE_WEIGHTED_AVG = {} WHERE REPORT_ID = {} """.format(row['FINAL_SCORE'],row['REPORT_ID'])) 
    conn.commit()
    
conn.close()
    

In [73]:
# all_data[all_data['REPORT_ID']==39201]

In [74]:
config = ConfigParser()

config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [75]:
report_risk = pd.read_sql_query('''select REPORT_ID,coalesce(RISK_SCORE_WEIGHTED_AVG,0) as RISK_SCORE_WEIGHTED_AVG,coalesce(RISK_SCORE_QUERY,0) as RISK_SCORE_QUERY from  FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT with (nolock) ''',conn)

In [76]:
report_risk['RISK_SCORE_WEIGHTED_AVG'].fillna(0,inplace = True)

# FINAL RISK SCORE CALCULATION.

In [77]:
report_risk['RISK_SCORE'] = (report_risk['RISK_SCORE_QUERY']**2 + report_risk['RISK_SCORE_WEIGHTED_AVG']**2 )/(report_risk['RISK_SCORE_QUERY'] + report_risk['RISK_SCORE_WEIGHTED_AVG'] )

In [78]:
report_risk['RISK_SCORE'].fillna(0,inplace = True)

In [79]:
report_risk['RISK_SCORE'] = report_risk['RISK_SCORE'].apply(lambda x: round_func(x)) 

In [80]:
# report_risk['RISK_SCORE'] = round_func(report_risk['RISK_SCORE'])

In [81]:
report_risk

REPORT_ID  RISK_SCORE_WEIGHTED_AVG  RISK_SCORE_QUERY  RISK_SCORE
0       100712                        9                 9           9
1       102012                        7                 9           8
2       102013                        7                 9           8
3       103112                        6                 0           6
4       103216                        6                 0           6
..         ...                      ...               ...         ...
177     135112                        7                10           9
178     135712                        7                 9           8
179     135812                        6                 0           6
180     135814                        7                10           9
181     138712                        9                10          10

[182 rows x 4 columns]

In [82]:
auto_high = pd.read_sql_query('''select REPORT_ID from Fincore_Analytics.RISK_REPORT_RULES A
inner join Fincore_Analytics.RISK_LKP_RULES B 
on A.RULE_NO = B.RULE_NO
where B.PERSON_TYPE = 'Report' and B.RISK_TYPE = 'Auto High Rules '  ''',conn)

In [83]:
auto_high = auto_high['REPORT_ID'].to_list()

In [84]:
report_risk = report_risk[~report_risk['REPORT_ID'].isin(auto_high)]

In [85]:
# UPDATING FINAL SCORE IN RISK_REPORT.
for ind,row in report_risk.iterrows():
#     print(row['BATCH_ID'])
    print("""UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = {} WHERE REPORT_ID = {} """.format(row['RISK_SCORE'],row['REPORT_ID'])) 

    cursor.execute("""UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = {} WHERE REPORT_ID = {} """.format(row['RISK_SCORE'],row['REPORT_ID'])) 
    conn.commit()
    
    

UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 9 WHERE REPORT_ID = 100712 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 8 WHERE REPORT_ID = 102012 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 8 WHERE REPORT_ID = 102013 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 6 WHERE REPORT_ID = 103112 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 6 WHERE REPORT_ID = 103216 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 6 WHERE REPORT_ID = 103217 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 5 WHERE REPORT_ID = 103312 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 5 WHERE REPORT_ID = 103313 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 9 WHERE REPORT_ID = 104014 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 8 WHERE REPORT_ID = 104112 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 9 WHE

UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 6 WHERE REPORT_ID = 116820 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 5 WHERE REPORT_ID = 116915 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 5 WHERE REPORT_ID = 116916 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 5 WHERE REPORT_ID = 116917 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 5 WHERE REPORT_ID = 116918 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 5 WHERE REPORT_ID = 116919 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 5 WHERE REPORT_ID = 116920 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 6 WHERE REPORT_ID = 116921 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 9 WHERE REPORT_ID = 118112 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 9 WHERE REPORT_ID = 118113 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 6 WHE

In [86]:
store -r no_of_records

In [87]:
store -r job_id

In [88]:
cursor.execute('''UPDATE FINCORE_DB.FINCORE_ANALYTICS.AUDIT_INTGN SET JOB_END_DATE = GETDATE() , JOB_STATUS = 'Success', no_of_record = {}  where job_run_id = {} '''.format(no_of_records,job_id))
conn.commit()

In [89]:
conn.close()